In [1]:
# -*- coding: utf-8 -*-
"""
Created on Mon Nov 15 17:16:43 2021

@author: Daniel Katusele
"""

from dolfin import *
import numpy as np
import math
import os
from mshr import *
from time import *

# Start time
st_time = time()

mesh = Mesh()
mvc = MeshValueCollection("size_t", mesh, mesh.topology().dim())
with XDMFFile("mesh3_V4.xdmf") as infile:
    infile.read(mesh)

# Geometric parameters
th = 0.2    # Thickness
rad = 1.35   # Radius to applied voltage
offset = 5.0


# Define Space
M = FunctionSpace(mesh, 'CG', 1)
V = VectorFunctionSpace(mesh, 'CG', 1)
T = TensorFunctionSpace(mesh, 'CG', 1)

W1 = VectorElement("CG", mesh.ufl_cell(), 2)
W2 = FiniteElement("CG", mesh.ufl_cell(), 1)
W = FunctionSpace(mesh, MixedElement([W1, W2]))
u_ = TrialFunction(W)
u, p = split(u_)
v_ = TestFunction(W)
v, q = split(v_)

phi, psi = TrialFunction(M), TestFunction(M)

phi_o, w = Function(M), Function(W)
phi_n, w2 = Function(M), Function(W)





# Mark boundary subdomians
tol = 1e-5
boundary_markers = MeshFunction('size_t', mesh, mesh.topology().dim()-1, 0)

boundary_markers.set_all(0)

def Dirichlet(x):
    return near(x[0], 0.0, tol) and near(x[1], 0.0, tol) and near(x[2], 0.0, tol)

class BoundarySide(SubDomain):
    def inside(self, x, on_boundary):
        r = math.sqrt(x[0]**2 + x[1]**2)
        return near(r, 1.5, 0.005) and on_boundary

class BoundaryTB(SubDomain):
    def inside(self, x, on_boundary):
        return (near(x[2], 0.0, tol) or near(x[2], th, tol)) and on_boundary

# The first sector (0 - 45 deg)
class BoundaryTop1(SubDomain):
    def inside(self, x, on_boundary):
        r = math.sqrt(x[0]**2 + x[1]**2)
        alpha = math.atan2(x[1],x[0])*180/math.pi
        if alpha < 0.0:
            alpha = alpha + 360.0
        return r >= rad and near(x[2], th, tol) and alpha > 0.0 + offset + tol and  alpha < 45.0 - offset - tol and on_boundary

# The 2nd sector (45 - 90 deg)
class BoundaryTop2(SubDomain):
    def inside(self, x, on_boundary):
        r = math.sqrt(x[0]**2 + x[1]**2)
        alpha = math.atan2(x[1],x[0])*180/math.pi
        if alpha < 0.0:
            alpha = alpha + 360.0
        return r >= rad and near(x[2], th, tol) and alpha > 45.0 + offset + tol and  alpha < 90.0 - offset - tol and on_boundary

# The sector (90 - 135 deg)
class BoundaryTop3(SubDomain):
    def inside(self, x, on_boundary):
        r = math.sqrt(x[0]**2 + x[1]**2)
        alpha = math.atan2(x[1],x[0])*180/math.pi
        if alpha < 0.0:
            alpha = alpha + 360.0
        return r >= rad and near(x[2], th, tol) and alpha > 90.0 + offset + tol and  alpha < 135.0 - offset - tol and on_boundary

# The sector (135 - 180 deg)
class BoundaryTop4(SubDomain):
    def inside(self, x, on_boundary):
        r = math.sqrt(x[0]**2 + x[1]**2)
        alpha = math.atan2(x[1],x[0])*180/math.pi
        if alpha < 0.0:
            alpha = alpha + 360.0
        return r >= rad and near(x[2], th, tol) and alpha > 135.0 + offset + tol and  alpha < 180.0 - offset - tol and on_boundary

# The first sector (180 - 225 deg)
class BoundaryTop5(SubDomain):
    def inside(self, x, on_boundary):
        r = math.sqrt(x[0]**2 + x[1]**2)
        alpha = math.atan2(x[1],x[0])*180/math.pi
        if alpha < 0.0:
            alpha = alpha + 360.0
        return r >= rad and near(x[2], th, tol) and alpha > 180.0 + offset + tol and  alpha < 225.0 - offset - tol and on_boundary

# The 2nd sector (225 - 270 deg)
class BoundaryTop6(SubDomain):
    def inside(self, x, on_boundary):
        r = math.sqrt(x[0]**2 + x[1]**2)
        alpha = math.atan2(x[1],x[0])*180/math.pi
        if alpha < 0.0:
            alpha = alpha + 360.0
        return r >= rad and near(x[2], th, tol) and alpha > 225.0 + offset + tol and  alpha < 270.0 - offset - 2*tol and on_boundary

# The first sector (270 - 315 deg)
class BoundaryTop7(SubDomain):
    def inside(self, x, on_boundary):
        r = math.sqrt(x[0]**2 + x[1]**2)
        alpha = math.atan2(x[1],x[0])*180/math.pi
        if alpha < 0.0:
            alpha = alpha + 360.0
        return r >= rad and near(x[2], th, tol) and alpha > 270.0 + offset + tol and  alpha < 315.0 - offset - 2*tol and on_boundary

# The 2nd sector (315 - 360 deg)
class BoundaryTop8(SubDomain):
    def inside(self, x, on_boundary):
        r = math.sqrt(x[0]**2 + x[1]**2)
        alpha = math.atan2(x[1],x[0])*180/math.pi
        if alpha < 0.0:
            alpha = alpha + 360.0
        return r >= rad and near(x[2], th, tol) and alpha > 315.0 + offset + tol and  alpha < 360.0 - offset - tol and on_boundary
 
# The sector (0 - 180 deg)
class BoundaryBot1(SubDomain):
    def inside(self, x, on_boundary):
        r = math.sqrt(x[0]**2 + x[1]**2)
        alpha = math.atan2(x[1],x[0])*180/math.pi
        if alpha < 0.0:
            alpha = alpha + 360.0
        return r >= rad and near(x[2], 0.0, tol) and alpha > 0.0 + offset + tol and alpha < 45 - offset - tol and on_boundary

# The sector (45 - 90 deg)
class BoundaryBot2(SubDomain):
    def inside(self, x, on_boundary):
        r = math.sqrt(x[0]**2 + x[1]**2)
        alpha = math.atan2(x[1],x[0])*180/math.pi
        if alpha < 0.0:
            alpha = alpha + 360.0
        return r >= rad and near(x[2], 0.0, tol) and alpha > 45.0 + offset + tol and alpha < 90.0 - offset - tol and on_boundary

        # The sector (90 - 135 deg)
class BoundaryBot3(SubDomain):
    def inside(self, x, on_boundary):
        r = math.sqrt(x[0]**2 + x[1]**2)
        alpha = math.atan2(x[1],x[0])*180/math.pi
        if alpha < 0.0:
            alpha = alpha + 360.0
        return r >= rad and near(x[2], 0.0, tol) and alpha > 90.0 + offset + tol and alpha < 135.0 - offset - tol and on_boundary

# The sector (135 - 180 deg)
class BoundaryBot4(SubDomain):
    def inside(self, x, on_boundary):
        r = math.sqrt(x[0]**2 + x[1]**2)
        alpha = math.atan2(x[1],x[0])*180/math.pi
        if alpha < 0.0:
            alpha = alpha + 360.0
        return r >= rad and near(x[2], 0.0, tol) and alpha > 135.0 + offset + tol and alpha < 180.0 - offset - tol and on_boundary

# The sector (180 - 225 deg)
class BoundaryBot5(SubDomain):
    def inside(self, x, on_boundary):
        r = math.sqrt(x[0]**2 + x[1]**2)
        alpha = math.atan2(x[1],x[0])*180/math.pi
        if alpha < 0.0:
            alpha = alpha + 360.0
        return r >= rad and near(x[2], 0.0, tol) and alpha > 180.0 + offset + tol and alpha < 225.0 - offset - tol and on_boundary

# The sector (225 - 270 deg)
class BoundaryBot6(SubDomain):
    def inside(self, x, on_boundary):
        r = math.sqrt(x[0]**2 + x[1]**2)
        alpha = math.atan2(x[1],x[0])*180/math.pi
        if alpha < 0.0:
            alpha = alpha + 360.0
        return r >= rad and near(x[2], 0.0, tol) and alpha > 225.0 + offset + tol and alpha < 270.0 - offset - tol and on_boundary

        # The sector (270 - 315 deg)
class BoundaryBot7(SubDomain):
    def inside(self, x, on_boundary):
        r = math.sqrt(x[0]**2 + x[1]**2)
        alpha = math.atan2(x[1],x[0])*180/math.pi
        if alpha < 0.0:
            alpha = alpha + 360.0
        return r >= rad and near(x[2], 0.0, tol) and alpha > 270.0 + offset + tol and alpha < 315.0 - offset - tol and on_boundary

# The sector (315 - 360 deg)
class BoundaryBot8(SubDomain):
    def inside(self, x, on_boundary):
        r = math.sqrt(x[0]**2 + x[1]**2)
        alpha = math.atan2(x[1],x[0])*180/math.pi
        if alpha < 0.0:
            alpha = alpha + 360.0
        return r >= rad and near(x[2], 0.0, tol) and alpha > 315.0 + offset + tol and alpha < 360.0 - offset - tol and on_boundary


b_top = BoundaryTB()
b_top.mark(boundary_markers, 1)

b_top2 = BoundaryTop2()
b_top2.mark(boundary_markers, 2)

b_top3 = BoundaryTop3()
b_top3.mark(boundary_markers, 3)

b_top4 = BoundaryTop4()
b_top4.mark(boundary_markers, 4)

b_top5 = BoundaryTop5()
b_top5.mark(boundary_markers, 5)

b_top6 = BoundaryTop6()
b_top6.mark(boundary_markers, 6)

b_top7 = BoundaryTop7()
b_top7.mark(boundary_markers, 7)

b_top8 = BoundaryTop8()
b_top8.mark(boundary_markers, 8)

b_bot = BoundaryBot1()
b_bot.mark(boundary_markers, 9)

b_bot2 = BoundaryBot2()
b_bot2.mark(boundary_markers, 10)

b_bot3 = BoundaryBot3()
b_bot3.mark(boundary_markers, 11)

b_bot4 = BoundaryBot4()
b_bot4.mark(boundary_markers, 12)

b_bot5 = BoundaryBot5()
b_bot5.mark(boundary_markers, 13)

b_bot6 = BoundaryBot6()
b_bot6.mark(boundary_markers, 14)

b_bot7 = BoundaryBot7()
b_bot7.mark(boundary_markers, 15)

b_bot8 = BoundaryBot8()
b_bot8.mark(boundary_markers, 16)

b_bot8 = BoundaryTop1()
b_bot8.mark(boundary_markers, 17)

File("Boundary_C.pvd") << boundary_markers
# Collect Dirichlet conditions
bcs2 = []
bcs1 = []
u_D = Constant((0.0, 0.0, 0.0))  # Zero displacement
volt1 = Expression('volt', degree= 1, volt=0.0)
volt2 = Expression('volt', degree= 1, volt=0.0)
volt3 = Expression('volt', degree= 1, volt=0.0)
volt4 = Expression('volt', degree= 1, volt=0.0)
volt5 = Expression('volt', degree= 1, volt=0.0)
volt6 = Expression('volt', degree= 1, volt=0.0)
volt7 = Expression('volt', degree= 1, volt=0.0)
volt8 = Expression('volt', degree= 1, volt=0.0)
# VOLT2 = Expression('volt', degree= 1, volt=0.0001)
# VOLT3 = Expression('-volt', degree= 1, volt=0.0001)
bcs2.append(DirichletBC(W.sub(0), u_D, Dirichlet, method="pointwise"))


bcs1.append(DirichletBC(M, volt1, boundary_markers, 17))
bcs1.append(DirichletBC(M, volt2, boundary_markers, 2))
bcs1.append(DirichletBC(M, volt3, boundary_markers, 3))
bcs1.append(DirichletBC(M, volt4, boundary_markers, 4))
bcs1.append(DirichletBC(M, volt5, boundary_markers, 5))
bcs1.append(DirichletBC(M, volt6, boundary_markers, 6))
bcs1.append(DirichletBC(M, volt7, boundary_markers, 7))
bcs1.append(DirichletBC(M, volt8, boundary_markers, 8))

def energy(phi, u, p):
    # Kinematics
    I = Identity(u.geometric_dimension())  # Identity tensor
    F = I + grad(u)  # Deformation gradient
    C = F.T * F  # Right Cauchy-Green tensor

    # Invariants of deformation tensors
    Ic = tr(C)
    IIc = 0.5 * (tr(C) * tr(C) - tr(C * C))

    FinvT = inv(F).T
    E = -grad(phi)

    # Energy
    engy = 0.5 * ((Ic - 3.0) + gamma*(IIc - 3.0))*dx + 0.5 * det(F) * dot(FinvT * E, FinvT * E)*dx - p * (det(F) - 1.0)*dx

    return engy, F


# Redefine boundary integration measure
ds = Measure('ds', domain=mesh, subdomain_data=boundary_markers)

# Constant variables
t0 = Constant((0.0, 0.0, 0.0))   # Zero traction
normal = FacetNormal(mesh)
incDLoad = 0.6                   # Load increament
DLoad = 4.5                 # Final load
Voltge = 5.e-3

incAngle = math.pi/100.0                   # Voltage increament
MaxAngle = math.pi                   # Voltage load

# Elasticity parameters
gamma = Constant(0.3)


# Define output file 
fName = input("Enter file name: ")
print('File name: ', fName + '.xdmf')
fileName = fName + '.xdmf'
fileName2 = fName + '.h5'
# try:
#     os.remove(fileName)
#     os.remove(fileName2)
# except OSError:
#     pass
fileResults = XDMFFile(fileName)
fileResults.parameters["flush_output"] = True
fileResults.parameters["functions_share_mesh"] = True

print('File name= ', fileName)
print('-------------------')
# Initialize iteration parameters
frame = 0         # Frame tracking parameter
j = 0.01          # Applied load at the start of the loop
k = 0.001         # Applied voltage at the start of the loop
l = 0
i=1               # Iteration tracking parameter


# Apply dead load incrementally
while j <= DLoad:
    print("---------- Dead Load Iteration: %i ------------" % (i))
    
    if j > 3.5 and j < 3.85:
        incDLoad = 0.05
    elif j >= 3.85 and j < 3.88:
        incDLoad = 0.01
        volt1.volt = Voltge*math.sin(l - 3*math.pi/4)
        volt2.volt = Voltge*math.sin(l - math.pi/2)
        volt3.volt = Voltge*math.sin(l - math.pi/4)
        volt4.volt = Voltge*math.sin(l)
        volt5.volt = Voltge*math.sin(l + math.pi/4)
        volt6.volt = Voltge*math.sin(l + math.pi/2)
        volt7.volt = Voltge*math.sin(l + 3*math.pi/4)
        volt8.volt = Voltge*math.sin(l + math.pi)
    elif j > 3.88 and j < 3.902:
        incDLoad = 0.001
    elif j > 3.902:
        incDLoad = 0.1


    print('Load: ', j)
    # Traction
    t = Constant(j) * normal
    t2 = Constant(j) * normal


    # Define variational problem
    '''
    p_disp = NonlinearVariationalProblem(Frm2, w2, bcs2, J=J)
    solver_d = NonlinearVariationalSolver(p_disp)


    prm = solver_d.parameters
    prm["nonlinear_solver"] = "newton"  # default. could be "snes"
    prm["newton_solver"]['linear_solver'] = 'petsc'
    prm["newton_solver"]["absolute_tolerance"] = 1E-9
    prm["newton_solver"]["relative_tolerance"] = 1.0E-5
    prm["newton_solver"]["maximum_iterations"] = 100
    prm["newton_solver"]["relaxation_parameter"] = 1.0
    prm["newton_solver"]['krylov_solver']['nonzero_initial_guess'] = True


    #solver_d.solve()
    '''
    err = 1
    iter = 1
    tolce = 1.e-3
    while err > tolce:
        # Total energy
        u_o, p_o = split(w)

        [Engy, F] = energy(phi_n, u_o, p_o)

        # Laplace equation
        Frm = det(F) * inner(inv(F).T * grad(phi), inv(F).T * grad(psi))*dx 

        p_phi = LinearVariationalProblem(lhs(Frm), rhs(Frm), phi_n, bcs1)
        solver_p = LinearVariationalSolver(p_phi)
        solver_p.solve()

        # Mechanical equation
        T_engy = Engy - dot(t,u_o)*ds(0) - dot(t0,u_o)*(ds(2) + ds(3)+ ds(4) + ds(5) + ds(6) + ds(7) \
                + ds(8) + ds(9) + ds(10) + ds(11) + ds(12) + ds(13) + ds(14) + ds(15) + ds(16) + ds(17)) \
            - det(F) * dot( inv(F.T*F) * grad(phi_n), normal) * phi_n * (ds(2) + ds(3)+ ds(4) + ds(5) + ds(6) + ds(7) \
                + ds(8) + ds(9) + ds(10) + ds(11) + ds(12) + ds(13) + ds(14) + ds(15) + ds(16) + ds(17))

        Frm2 = derivative(T_engy, w, v_)

        solve(Frm2 == 0, w, bcs2, solver_parameters={"newton_solver":
                                                {'relative_tolerance': 1.0e-5,'absolute_tolerance': 1.0e-9, 'maximum_iterations': 100,
                                                'relaxation_parameter': 1.0, 'linear_solver': 'petsc'}},
            form_compiler_parameters = {'cpp_optimize': True})


        err_u = errornorm(w2.sub(0), w.sub(0), norm_type = 'l2',mesh = mesh)
        err_phi = errornorm(phi_n, phi_o, norm_type = 'l2',mesh = mesh)
        err = max(err_u,err_phi)

        w2.assign(w)
        phi_o.assign(phi_n)

        iter = iter + 1
        print('Error: ', err)

    # Post-processing

    u_n, p_n = split(w2)
    I = Identity(u_n.geometric_dimension())  # Identity tensor
    F_p = I + grad(u_n)  # Deformation gradient
    C = F_p.T * F_p  # Right Cauchy-Green tensor

    # Invariants of deformation tensors
    Ic = tr(C)

    FinvT = inv(F_p).T

    E = -grad(phi_o)
    
    C_stress = (1.0/det(F_p)) * ((1.0 + gamma*Ic) * F_p * F_p.T - gamma * F_p * F_p.T * F_p * F_p.T) - p_n * I \
            + (outer(FinvT * E, FinvT * E) - 0.5 * dot(FinvT * E, FinvT * E) * I)

    
    if i % 1.0 == 0.0:
        frame += 1
        disp = Function(V, name='Displacement')
        disp.assign(project(u_n, V))
        fileResults.write(disp, frame)

        lgr = Function(M, name='Lgr Mult')
        lgr.assign(project(p_n, M))
        fileResults.write(lgr, frame)

        voltage = Function(M, name='Voltage')
        voltage.assign(project(phi_n, M))
        fileResults.write(voltage, frame)

        defgrad = Function(T, name='DefGrad')
        defgrad.assign(project(F_p, T))
        fileResults.write(defgrad, frame)

        el_f = Function(V, name='Electric field')
        el_f.assign(project(-grad(phi_n), V))
        fileResults.write(el_f, frame)

    j = j + incDLoad

    if j > DLoad:
        DLoad = j - incDLoad
    w.assign(w)
    w2.assign(w2)
    i = i + 1

print('Time: ', strftime("%H:%M:%S", gmtime(time() - st_time))) 

# Apply voltage incremetally
i = 1

while l <= MaxAngle:
    print("---------- Voltage Iteration: %i ------------" % (i))
    
    #parameters['linear_algebra_backend'] = 'PETSc'
    parameters["form_compiler"]["cpp_optimize"] = True
    parameters["form_compiler"]["quadrature_rule"] = 'auto'
    #parameters["form_compiler"]["quadrature_degree"] = 6
    # Traction
    t = Constant(DLoad) * normal
    t2 = Constant(DLoad) * normal

    
    volt1.volt = Voltge*math.sin(l - 3*math.pi/4)
    volt2.volt = Voltge*math.sin(l - math.pi/2)
    volt3.volt = Voltge*math.sin(l - math.pi/4)
    volt4.volt = Voltge*math.sin(l)
    volt5.volt = Voltge*math.sin(l + math.pi/4)
    volt6.volt = Voltge*math.sin(l + math.pi/2)
    volt7.volt = Voltge*math.sin(l + 3*math.pi/4)
    volt8.volt = Voltge*math.sin(l + math.pi)

    err = 1
    iter = 1
    tolce = 1.e-4
    while err > tolce:
        # Total energy
        u_o, p_o = split(w)

        [Engy, F] = energy(phi_n, u_o, p_o)

        # Laplace equation
        Frm = det(F) * inner(inv(F).T * grad(phi), inv(F).T * grad(psi))*dx 

        p_phi = LinearVariationalProblem(lhs(Frm), rhs(Frm), phi_n, bcs1)
        solver_p = LinearVariationalSolver(p_phi)
        solver_p.solve()

        # Mechanical equation
        T_engy = Engy - dot(t,u_o)*ds(0) - dot(t0,u_o)*(ds(2) + ds(3)+ ds(4) + ds(5) + ds(6) + ds(7) \
                + ds(8) + ds(9) + ds(10) + ds(11) + ds(12) + ds(13) + ds(14) + ds(15) + ds(16) + ds(17)) \
            - dot(det(F) * inv(F.T*F) * grad(phi_n), normal) * phi_n * (ds(2) + ds(3)+ ds(4) + ds(5) + ds(6) + ds(7) \
                + ds(8) + ds(9) + ds(10) + ds(11) + ds(12) + ds(13) + ds(14) + ds(15) + ds(16) + ds(17))

        Frm2 = derivative(T_engy, w, v_)

        solve(Frm2 == 0, w, bcs2, solver_parameters={"newton_solver":
                                                {'relative_tolerance': 1.0e-6,'absolute_tolerance': 1.0e-9, 'maximum_iterations': 100,
                                                'relaxation_parameter': 1.0, 'linear_solver': 'petsc'}},
            form_compiler_parameters = {'cpp_optimize': True})

        u_o, p_o = split(w)
        u_n, p_n = split(w2)

        err_u = errornorm(w2.sub(0), w.sub(0), norm_type = 'l2', mesh = mesh)
        err_phi = errornorm(phi_n, phi_o, norm_type = 'l2', mesh = mesh)
        err = max(err_u,err_phi)

        w2.assign(w)
        phi_o.assign(phi_n)

        iter = iter + 1
        print('Error: ', err)

     # Post-processing

    u_n, p_n = split(w2)
    I = Identity(u_n.geometric_dimension())  # Identity tensor
    F_p = I + grad(u_n)  # Deformation gradient
    C = F_p.T * F_p  # Right Cauchy-Green tensor

    # Invariants of deformation tensors
    Ic = tr(C)

    FinvT = inv(F_p).T

    E = -grad(phi_o)
    
    C_stress = (1.0/det(F_p)) * ((1.0 + gamma*Ic) * F_p * F_p.T - gamma * F_p * F_p.T * F_p * F_p.T) - p_n * I \
            + (outer(FinvT * E, FinvT * E) - 0.5 * dot(FinvT * E, FinvT * E) * I)

    if i % 1.0 == 0.0:
        frame += 1
        u_n, p_n = split(w2)
        disp = Function(V, name='Displacement')
        disp.assign(project(u_n, V))
        fileResults.write(disp, frame)

        lgr = Function(M, name='Lgr Mult')
        lgr.assign(project(p_n, M))
        fileResults.write(lgr, frame)

        voltage = Function(M, name='Voltage')
        voltage.assign(project(phi_n, M))
        fileResults.write(voltage, frame)

        defgrad = Function(T, name='DefGrad')
        defgrad.assign(project(F_p, T))
        fileResults.write(defgrad, frame)

        el_f = Function(V, name='Electric field')
        el_f.assign(project(-grad(phi_n), V))
        fileResults.write(el_f, frame)

    l = l + incAngle

    w.assign(w)
    w2.assign(w2)
    i = i + 1


print('Initial Volume: ', 1.4137)

print('Time: ', strftime("%H:%M:%S", gmtime(time() - st_time)))

print('Successfully completed')





File name:  Electromechanical3.xdmf
File name=  Electromechanical3.xdmf
-------------------
---------- Dead Load Iteration: 1 ------------
Load:  0.01
Solving linear variational problem.
No Jacobian form specified for nonlinear variational problem.
Differentiating residual form F to obtain Jacobian J = F'.
Solving nonlinear variational problem.
  Newton iteration 0: r (abs) = 3.509e-01 (tol = 1.000e-09) r (rel) = 1.000e+00 (tol = 1.000e-05)
  Newton iteration 1: r (abs) = 2.208e-03 (tol = 1.000e-09) r (rel) = 6.293e-03 (tol = 1.000e-05)
  Newton iteration 2: r (abs) = 3.256e-06 (tol = 1.000e-09) r (rel) = 9.280e-06 (tol = 1.000e-05)
  Newton solver finished in 2 iterations and 2 linear solver iterations.
*** Warning: Degree of exact solution may be inadequate for accurate result in errornorm.
*** Warning: Degree of exact solution may be inadequate for accurate result in errornorm.
Error:  0.0016577521952976654
Solving linear variational problem.
No Jacobian form specified for nonlinear